# Train

In [2]:
import numpy as np
import pandas as pd 
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import csv
import tensorflow as tf
from PIL import Image
import os
import keras
import sys
import random
import shutil
import json
import skimage.io as io

# Import architectures
from keras.applications import xception

from keras.preprocessing import image
from tensorflow.keras.preprocessing import image as im
import time
import datetime

from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout

from scipy.spatial.transform import Rotation as R # for quaternion
import skimage.transform as trans

In [3]:
# Train phase

tf.keras.backend.set_learning_phase(1) 

In [4]:
# Choose GPU

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
# Make test dataset

num_train_photo = len(os.listdir(train_dir))
while len(os.listdir(test_dir)) < 0.1 * num_train_photo:
    photo = random.choice(os.listdir(train_dir))
    shutil.move(train_dir + photo, test_dir + photo)

In [ ]:
# Create 256x256 photos with with black stripes around the edges

test = "/home/linnik/Datasets/Baidu/train_crops/NEW/"

for photo in os.listdir(test_dir):
    img = Image.open(test_dir + photo)
    w, h = img.size

    if w == h:
        new_w = 256
        new_h = 256

        new_img = img.resize((new_w, new_h))
        colored_PIL_image = new_img

    if w > h:
        new_w = 256
        new_h = int((new_w * h)/w)

        new_img = img.resize((new_w, new_h))
        x = np.array(new_img)

        new = np.zeros((1, 256, 3))
        
        print(new.shape, x.shape, type(new), type(x))

        while True:
            x = np.append(new, x, axis = 0)
            if x.shape[0] == 256:
                break
            x = np.append(x, new, axis = 0)
            if x.shape[0] == 256:
                break

        colored_PIL_image = Image.fromarray(x.astype(np.uint8))


    if w < h:
        new_h = 256
        new_w = int((new_h * w)/h)

        new_img = img.resize((new_w, new_h))
        x = np.array(new_img)

        new = np.zeros((256, 1, 3))

        while True:
            x = np.append(new, x, axis = 1)
            if x.shape[1] == 256:
                break
            x = np.append(x, new, axis = 1)
            if x.shape[1] == 256:
                break

        colored_PIL_image = Image.fromarray(x.astype(np.uint8))

    colored_PIL_image.save(test + photo)
    break

In [ ]:
with open(annotations_dir) as json_data:
    data = json.load(json_data)
    
print(data.keys())

categories = pd.DataFrame.from_dict(data["categories"])
images = pd.DataFrame.from_dict(data["images"])
annotations = pd.DataFrame.from_dict(data["annotations"])

In [ ]:
categories

In [ ]:
annotations

In [ ]:
images

In [ ]:
def rotation_to_quaternion(rotation):
    rotation = R.from_euler("YXZ", rotation)
    q = rotation.as_quat()
    return q

def quaternion_to_rotation(q):
    rotation = R.from_quat(q)
    ypr = rotation.as_euler("YXZ")
    return ypr

In [ ]:
angle_train = pd.DataFrame(columns = ["file_name", "yaw", "pitch", "roll", "q_0", "q_1", "q_2", "q_3"])
angle_test = pd.DataFrame(columns = ["file_name", "yaw", "pitch", "roll", "q_0", "q_1", "q_2", "q_3"])

In [ ]:
for photo in os.listdir(train_dir):
    id = int(images[images["file_name"] == photo]["id"])
    ang = list(annotations[annotations["image_id"] == id]["orientation"])[0]
    angle_train.loc[len(angle_train)] = [photo, ang[0], ang[1], ang[2], rotation_to_quaternion(ang)[0],
                                        rotation_to_quaternion(ang)[1], rotation_to_quaternion(ang)[2],
                                        rotation_to_quaternion(ang)[3]]
    
    print(len(angle_train))

In [ ]:
angle_train.to_csv("origin_train_angeles_and_quaternions.csv")

In [ ]:
for photo in os.listdir(test_dir):
    id = int(images[images["file_name"] == photo]["id"])
    ang = list(annotations[annotations["image_id"] == id]["orientation"])[0]
    angle_test.loc[len(angle_test)] = [photo, ang[0], ang[1], ang[2], rotation_to_quaternion(ang)[0],
                                        rotation_to_quaternion(ang)[1], rotation_to_quaternion(ang)[2],
                                        rotation_to_quaternion(ang)[3]]
    
    print(len(angle_test))

In [ ]:
angle_test.to_csv("origin_test_angeles_and_quaternions.csv")

In [5]:
angle_train = pd.read_csv("/home/linnik/Datasets/Baidu/train_crops/origin_train_angeles_and_quaternions.csv")
angle_test = pd.read_csv("/home/linnik/Datasets/Baidu/train_crops/origin_test_angeles_and_quaternions.csv")

In [6]:
print(angle_train.shape, angle_test.shape)

angle_test.head()

(44203, 9) (4912, 9)


,Unnamed: 0,file_name,yaw,pitch,roll,q_0,q_1,q_2,q_3
0,0,ID_5e319247e_18032.jpg,-3.117980,-0.147002,-3.06390,0.996445,-0.039595,-0.014617,-0.072917
1,1,ID_05aea87a6_1258.jpg,0.142895,-0.189191,3.02060,0.065241,0.098338,0.991580,0.053305
2,2,ID_b8de58ad8_36084.jpg,0.118927,-0.144334,3.08396,0.057175,0.073655,0.995344,0.024403
3,3,ID_19a5897e7_4866.jpg,-0.017885,-0.162836,3.09728,-0.010712,0.081107,0.996387,0.022807
4,4,ID_3f88af6f4_12007.jpg,0.285412,-0.148567,3.09430,0.140053,0.076794,0.987079,0.012787


In [7]:
# 1) Path to train, test datasets and to annotations file
train_dir = "/home/linnik/Datasets/Baidu/train_crops/origin/train/"
test_dir = "/home/linnik/Datasets/Baidu/train_crops/origin/test/"
#annotations_dir = "/home/linnik/Datasets/Baidu/train_crops/annotations.json"

# 2) Target width and height of input images, number of classes, number of train and test images, batch size, epochs
img_height, img_width = 128, 128
n_classes = len(os.listdir(train_dir))
nb_train_samples = len(os.listdir(train_dir))
nb_test_samples = len(os.listdir(test_dir))
batch_size = 32
epochs = 100


# 3) Name of file where will be saved trained model
# model_name = ".../Architecture_InputShape_NClasses_date_time.hdf5"
model_name = "/home/linnik/Nets/Regression_Xception_" + str(img_height) + "x" + str(img_width) + ".hdf5"


# 4) Compile parametrs
loss_ = 'mse'
optimizer_ = 'adam'
metrics_ = ['mse', "mae"]

In [8]:
print(nb_train_samples, nb_test_samples)

44203 4912


In [9]:
# Create and compile Keras model

def build_model_Xception():
    base_model = xception.Xception(include_top = False, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape = (img_height, img_width, 3),
                                      pooling = None)
  
    for layer in base_model.layers:
        layer.trainable = True
    
    # Add MLP to Xception base_model
    flat = Flatten()(base_model.output)
    dense1 = Dense(300, activation = "relu")(flat)
    dense2 = Dense(100, activation = "relu")(dense1)
    dense3 = Dense(4, activation = "sigmoid")(dense2)
    
    model = Model(inputs = base_model.input, outputs = dense3)
    return model

In [10]:
model = build_model_Xception()           
model.compile(loss = loss_, optimizer = optimizer_, metrics = metrics_)

In [9]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def resNetM(input_shape, classes):
    img_input = Input(shape=input_shape)

    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = AveragePooling2D((3, 3), name='avg_pool')(x)

    x = Flatten()(x)
    x = Dense(300, activation='relu', name='fc01')(x)
    x = Dense(100, activation='relu', name='fc02')(x)
    x = Dropout(0.3, name='dr01')(x)
    x = Dense(classes, activation='softmax', name='fc5')(x)

    inputs = img_input

    model = Model(inputs, x, name='resnetM')

    return model

In [10]:
model = resNetM((img_height, img_width, 3), 4)
model.compile(loss = loss_, optimizer = optimizer_, metrics = metrics_)









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [11]:
model.summary()

Model: "resnetM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           bn_conv1[0][0]                   
____________________________________________________________________________________________

In [12]:
def my_generator(photo_path, angle_df, batch_size, img_height, img_width):
    while True:
        for index, row in angle_train.iterrows():
            img = io.imread(train_dir + row["file_name"], as_gray=False)
            w, h = img.shape[:2]

            if w == h:
                new_w = img_width
                new_h = img_height
                img = trans.resize(img, (new_h, new_w, 3)) / 255
            if w > h:
                new_w = img_width
                new_h = int((new_w * h) / w)
                img = trans.resize(img, (new_h, new_w, 3)) / 255
                new = np.zeros((1, img_width, 3))
                while True:
                    x = np.append(new, img, axis = 0)
                    if img.shape[0] == img_height:
                        break
                    img = np.append(img, new, axis = 0)
                    if img.shape[0] == img_height:
                        break
            if w < h:
                new_h = img_height
                new_w = int((new_h * w) / h)
                img = trans.resize(img, (new_h, new_w, 3)) / 255
                new = np.zeros((img_height, 1, 3))
                while True:
                    img = np.append(new, img, axis = 1)
                    if img.shape[1] == img_width:
                        break
                    img = np.append(img, new, axis = 1)
                    if img.shape[1] == img_width:
                        break
                        
            img = np.reshape(img, (1,) + img.shape)
            
            q_0 = (np.array(row["q_0"]).reshape((1,1)) + 1.0) * 0.5
            q_1 = (np.array(row["q_1"]).reshape((1,1)) + 1.0) * 0.5
            q_2 = (np.array(row["q_2"]).reshape((1,1)) + 1.0) * 0.5
            q_3 = (np.array(row["q_3"]).reshape((1,1)) + 1.0) * 0.5

            
            angle = np.concatenate((q_0, np.concatenate((q_1, np.concatenate((q_2, q_3), axis = 1)), axis = 1)), axis = 1)

            if (index % batch_size == 0):
                img_batch = img
                angle_batch = angle
            else:
                img_batch = np.concatenate((img_batch, img), axis = 0)
                angle_batch = np.concatenate((angle_batch, angle), axis = 0)
                
            if (index + 1) % batch_size == 0:
                yield (img_batch, angle_batch)

In [13]:
train_generator = my_generator(train_dir, angle_train, batch_size = batch_size, img_height = img_height, img_width = img_width)
test_generator = my_generator(test_dir, angle_test, batch_size = batch_size, img_height = img_height, img_width = img_width)

In [14]:
# Callbacks

early_stop = EarlyStopping(monitor = 'val_loss', patience = 8, verbose = 1, min_delta = 1e-4)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 4, verbose = 1, min_delta = 1e-4)
check = ModelCheckpoint(filepath = model_name, monitor = "val_loss", save_best_only = True)

callbacks_list = [early_stop, reduce_lr, check]

In [ ]:
%%time

# Train model

model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = test_generator,
    validation_steps = nb_test_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size)

Epoch 1/100
1381/1381 [==============================] - 770s 558ms/step - loss: 0.2528 - mean_squared_error: 0.2528 - mean_absolute_error: 0.4403 - val_loss: 0.2535 - val_mean_squared_error: 0.2535 - val_mean_absolute_error: 0.4410
Epoch 2/100
1345/1381 [============================>.] - ETA: 13s - loss: 0.2528 - mean_squared_error: 0.2528 - mean_absolute_error: 0.4403

In [ ]:
# kill kernel
os._exit(0)

# Test

In [14]:
# Test phase

tf.keras.backend.set_learning_phase(0)

In [15]:
# Load trained model

loaded_model = load_model(model_name)